In [1]:
import os
import pickle
import pandas as pd

In [2]:
from sklearn.metrics import log_loss, confusion_matrix, classification_report, roc_auc_score, precision_score, recall_score, f1_score, accuracy_score

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
INPUT_DIR = '../input/ieee-fraud-detection-dataprocessing'
data = {}
files = os.listdir(INPUT_DIR)
for file in files:
    if 'Xtrain' in file:
        data['Xtrain'] = INPUT_DIR + '/' + file
    if 'Ytrain' in file:
        data['Ytrain'] = INPUT_DIR + '/' + file
    if 'Xtest' in file:
        data['Xtest'] = INPUT_DIR + '/' + file
data

{'Xtest': '../input/ieee-fraud-detection-dataprocessing/Xtest-IEEE-CIS-Fraud-Detection.data',
 'Xtrain': '../input/ieee-fraud-detection-dataprocessing/Xtrain-IEEE-CIS-Fraud-Detection.data',
 'Ytrain': '../input/ieee-fraud-detection-dataprocessing/Ytrain-IEEE-CIS-Fraud-Detection.data'}

In [5]:
Xtrain = pickle.load(open(data['Xtrain'], 'rb'))
Xtrain.head()

,C1,C10,C11,C12,C13,C14,C2,C3,C4,C5,...,id_34_match_status:1,id_34_match_status:2,id_35_F,id_35_T,id_36_F,id_36_T,id_37_F,id_37_T,id_38_F,id_38_T
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0.000214,0.000000,0.000628,0.0,0.000343,0.0007,0.000176,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
2987001,0.000214,0.000000,0.000314,0.0,0.000343,0.0007,0.000176,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
2987002,0.000214,0.000000,0.000314,0.0,0.000343,0.0007,0.000176,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
2987003,0.000427,0.000000,0.000314,0.0,0.008568,0.0007,0.000878,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
2987004,0.000214,0.000307,0.000314,0.0,0.000343,0.0007,0.000176,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0


In [6]:
Ytrain = pickle.load(open(data['Ytrain'], 'rb'))
Ytrain.head()

TransactionID
2987000    0
2987001    0
2987002    0
2987003    0
2987004    0
Name: isFraud, dtype: int64

In [7]:
Xtest = pickle.load(open(data['Xtest'], 'rb'))
Xtest.head()

,C1,C10,C11,C12,C13,C14,C2,C3,C4,C5,...,id_34_match_status:1,id_34_match_status:2,id_35_F,id_35_T,id_36_F,id_36_T,id_37_F,id_37_T,id_38_F,id_38_T
TransactionID,,,,,,,,,,,,,,,,,,,,,
3663549,0.001281,0.0,0.001569,0.000314,0.039398,0.0042,0.001054,0.0,0.0,0.008598,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
3663550,0.000640,0.0,0.000314,0.000314,0.004112,0.0014,0.000351,0.0,0.0,0.000000,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
3663551,0.000427,0.0,0.000628,0.000000,0.007538,0.0014,0.000351,0.0,0.0,0.000000,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
3663552,0.001067,0.0,0.000628,0.000000,0.002399,0.0028,0.000351,0.0,0.0,0.002865,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
3663553,0.001281,0.0,0.001882,0.000000,0.004799,0.0042,0.001054,0.0,0.0,0.005730,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0


In [8]:
Xtrain, Xval, Ytrain, Yval = train_test_split(Xtrain, Ytrain, test_size=0.1)

In [9]:
from lightgbm import LGBMClassifier

In [10]:
model_df = pd.read_csv('models_params.csv') if 'models_params.csv' in os.listdir('./') else pd.DataFrame(columns=['model_name',
                                 'boosting_type',
                                 'n_estimators',
                                 'num_leaves',
                                 'objective',
                                 'learning_rate',
                                 'tree_learner',
                                 'reg_alpha',
                                 'reg_lamdba',
                                 'accuaracy_score',
                                 'log_loss',
                                 'roc_auc_score',
                                 'precision_score',
                                 'recall_score',
                                 'f1_socre',
                                 'confusion_matrix'], index=['model_name'])
def model_train_eval(model_no,
                     boosting_type,
                     n_estimators, 
                     num_leaves, 
                     learning_rate, 
                     objective, 
                     tree_learner, 
                     reg_alpha, 
                     reg_lambda):
    insert_dict = {}
    global model_df
    clf = LGBMClassifier(boosting_type=boosting_type, 
                     n_estimators=n_estimators, 
                     num_leaves=num_leaves, 
                     learning_rate=learning_rate, 
                     objective=objective, 
                     tree_learner=tree_learner,
                     reg_alpha=reg_alpha,
                     reg_lambda=reg_lambda,
                     silent=False)
    clf.fit(Xtrain, Ytrain)
    print('###################################################################################')
    print()
    print(str(clf))
    print()
    Ytrain_ = clf.predict(Xtrain)
    Yval_ = clf.predict(Xval)
    print('Training Results:')
    print('=======================================')
    print('Accuracy Score:', accuracy_score(Ytrain, Ytrain_))
    print()
    print('Log loss:', log_loss(Ytrain, Ytrain_))
    print()
    print('ROC_Accuracy_Score:', roc_auc_score(Ytrain, Ytrain_))
    print()
    print('Precision_Score:', precision_score(Ytrain, Ytrain_))
    print()
    print('Recall_Score:', recall_score(Ytrain, Ytrain_))
    print()
    print('F1_Score:', f1_score(Ytrain, Ytrain_))
    print()
    print('Confusion_Matrix:')
    print(confusion_matrix(Ytrain, Ytrain_))
    print()
    print('Classification Report:')
    print(classification_report(Ytrain, Ytrain_))
    print()
    print('Validation Results:')
    print('=======================================')
    print('Accuracy Score:', accuracy_score(Yval, Yval_))
    print()
    print('Log loss:', log_loss(Yval, Yval_))
    print()
    print('ROC_Accuracy_Score:', roc_auc_score(Yval, Yval_))
    print()
    print('Precision_Score:', precision_score(Yval, Yval_))
    print()
    print('Recall_Score:', recall_score(Yval, Yval_))
    print()
    print('F1_Score:', f1_score(Yval, Yval_))
    print()
    print('Confusion_Matrix:')
    print(confusion_matrix(Yval, Yval_))
    print()
    print('Classification Report:')
    print(classification_report(Yval, Yval_))
    print()
    filename = './models/'+'model_'+str(model_no)+'.model'
    insert_dict['model_name'] = 'model_'+str(model_no)+'.model'
    insert_dict['boosting_type'] = boosting_type
    insert_dict['n_estimators'] = n_estimators
    insert_dict['num_leaves'] = num_leaves
    insert_dict['objective'] = objective
    insert_dict['learning_rate'] = learning_rate
    insert_dict['tree_learner'] = tree_learner
    insert_dict['reg_alpha'] = reg_alpha
    insert_dict['reg_lamdba'] = reg_lambda
    insert_dict['accuaracy_score'] = accuracy_score(Yval, Yval_)
    insert_dict['log_loss'] = log_loss(Yval, Yval_)
    insert_dict['roc_auc_score'] = roc_auc_score(Yval, Yval_)
    insert_dict['precision_score'] = precision_score(Yval, Yval_)
    insert_dict['recall_score'] = recall_score(Yval, Yval_)
    insert_dict['f1_socre'] = f1_score(Yval, Yval_)
    insert_dict['confusion_matrix'] = confusion_matrix(Yval, Yval_)
    model_df = model_df.append(insert_dict, ignore_index=True)
    model_df.to_csv('./models_params.csv')
    print('Added to models_params.csv')
    print()
    pickle.dump(clf, open(filename, 'wb'))
    print('Saving to', filename)
    print()
    print('###################################################################################')
    print()
    print()

In [11]:
param_boosting_type = ['goss']
param_n_estimators = [600]
param_num_leaves = [600]
param_learning_rate = [0.01]
param_objective = ['cross_entropy']
param_tree_learner = ['voting']
param_reg_alpha = [0.01]
param_reg_lambda = [0.01]

In [12]:
model_no = 8
for n_estimators in param_n_estimators:
    for boosting_type in param_boosting_type:
        for num_leaves in param_num_leaves:
            for learning_rate in param_learning_rate:
                for objective in param_objective:
                    for tree_learner in param_tree_learner:
                        for reg_alpha in param_reg_alpha:
                            for reg_lambda in param_reg_lambda:
                                model_no += 1
                                model_train_eval(model_no,
                                                boosting_type,
                                                n_estimators,
                                                num_leaves,
                                                learning_rate,
                                                objective,
                                                tree_learner,
                                                reg_alpha,
                                                reg_lambda)

###################################################################################

LGBMClassifier(boosting_type='goss', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.01, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=600, n_jobs=-1, num_leaves=600,
               objective='cross_entropy', random_state=None, reg_alpha=0.01,
               reg_lambda=0.01, silent=False, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0,
               tree_learner='voting')

Training Results:
Accuracy Score: 0.9832497869909473

Log loss: 0.5785412874418073

ROC_Accuracy_Score: 0.9832472159822242

Precision_Score: 0.9767499096299829

Recall_Score: 0.990079558433155

F1_Score: 0.9833695650069787

Confusion_Matrix:
[[500604  12092]
 [  5090 507992]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0